# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, accuracy_score
import nltk
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
nltk.download(['punkt','wordnet'])
import re
import pickle
from bs4 import BeautifulSoup

[nltk_data] Downloading package punkt to /Users/suman/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /Users/suman/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [7]:
# load data from database
engine = create_engine('sqlite:///../data/disaster.db')
df = pd.read_sql_table('response',engine)
X = df['message']
Y = df.iloc[:,5:]
Y.fillna(0,inplace=True)

In [8]:
Y

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26355,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26357,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26358,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 2. Write a tokenization function to process your text data

In [9]:
def tokenize(text):
    #tokens = word_tokenize(text)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens=tokenizer.tokenize(t)
    return tokens

In [10]:
t='Haitians in U.S. watch, earthquake #news and worry http bit.ly 6PkPGv News #CNN favorited false in_reply_to_user_id null in_reply_to_status_id null in_reply_to_screen_name null source <a href= http ping.fm rel= nofollow >Ping.fm< a>'
#soup=BeautifulSoup(t,'html5lib')
#print(soup.get_text())
tokenize(t)

#tokenizer = RegexpTokenizer(r'\w+')
#tokenizer.tokenize(t)


['Haitians',
 'in',
 'U',
 'S',
 'watch',
 'earthquake',
 'news',
 'and',
 'worry',
 'http',
 'bit',
 'ly',
 '6PkPGv',
 'News',
 'CNN',
 'favorited',
 'false',
 'in_reply_to_user_id',
 'null',
 'in_reply_to_status_id',
 'null',
 'in_reply_to_screen_name',
 'null',
 'source',
 'a',
 'href',
 'http',
 'ping',
 'fm',
 'rel',
 'nofollow',
 'Ping',
 'fm',
 'a']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
def model_pipeline():
    pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier())),
    ])
    return pipeline

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)
np.any(np.isnan(Y))
model = []
model = model_pipeline()
model.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                                                                        ccp_alpha=0.0,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                   

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
Y_pred = model.predict(X_test)
category_names=list(Y.columns)    
# Calculate the accuracy for each of them.
for i in range(len(category_names)):
    print("Category:", category_names[i],"\n", classification_report(y_test.iloc[:, i].values, Y_pred[:, i]))
    print('Accuracy of %25s: %.2f' %(category_names[i], accuracy_score(y_test.iloc[:, i].values, Y_pred[:,i])))

/Users/suman/opt/miniconda3/envs/disaster_reponse_pipeline_env/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Category: related 
               precision    recall  f1-score   support

         0.0       0.00      0.00      0.00      1583
         1.0       0.75      1.00      0.86      4965
         2.0       0.00      0.00      0.00        42

    accuracy                           0.75      6590
   macro avg       0.25      0.33      0.29      6590
weighted avg       0.57      0.75      0.65      6590

Accuracy of                   related: 0.75
Category: request 
               precision    recall  f1-score   support

         0.0       0.83      1.00      0.91      5470
         1.0       0.00      0.00      0.00      1120

    accuracy                           0.83      6590
   macro avg       0.42      0.50      0.45      6590
weighted avg       0.69      0.83      0.75      6590

Accuracy of                   request: 0.83
Category: offer 
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00      6567
         1.0       0.00      0.00    

Category: weather_related 
               precision    recall  f1-score   support

         0.0       0.72      1.00      0.84      4763
         1.0       0.00      0.00      0.00      1827

    accuracy                           0.72      6590
   macro avg       0.36      0.50      0.42      6590
weighted avg       0.52      0.72      0.61      6590

Accuracy of           weather_related: 0.72
Category: floods 
               precision    recall  f1-score   support

         0.0       0.92      1.00      0.96      6055
         1.0       0.00      0.00      0.00       535

    accuracy                           0.92      6590
   macro avg       0.46      0.50      0.48      6590
weighted avg       0.84      0.92      0.88      6590

Accuracy of                    floods: 0.92
Category: storm 
               precision    recall  f1-score   support

         0.0       0.91      1.00      0.95      5967
         1.0       0.00      0.00      0.00       623

    accuracy                 

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [15]:
pickle.dump(model, open('../models/classifier.pkl', "wb"))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.